# Naïve pertubation of windforcing

Simple example of pertubation of windforcing in simulations, using only wind direction and wind speed as input.

## Setup and initialization

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys
import glob
import math as m

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS, CDKLM16, SimWriter, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

In [ ]:
def wind_u(wind_speed, wind_direction):
    return -wind_speed * m.sin(wind_direction * 0.01745329)

def wind_v(wind_speed, wind_direction):
    return -wind_speed * m.cos(wind_direction * 0.01745329)

u = wind_u(5.0, 0.0)
v = wind_v(5.0, 0.0)

print u
print v

## Make nc-file

In [ ]:
#Clean up old simulator if any:
if 'sim' in globals():
    sim.cleanUp()
    
#Centered in time, centered in space
nx = 50
ny = 50

dx = 4.0
dy = 4.0

width = nx * dx
height = ny * dy

dt = 0.1
g = 9.81
r = 0.0
A = 1
#f = 0.01
f = 0.0

#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=50, wind_speed=3.0, wind_direction=270.0)
#wind = Common.WindStressParams(type=99)

    

bcSettings = 2
ghosts = [1,1,1,1] # north, east, south, west
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    # Wall boundary conditions
elif (bcSettings == 2):
    # periodic boundary conditions
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # periodic north-south
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # periodic east-west
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C')*5 

# Bottom with a bump:
#h0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
#addBump(h0, nx, ny, dx, dy, 0.5, 0.5, 600, ghosts)
#h0 = (5.0 - h0*4.2)
#fig = plt.figure(figsize=(4,4))
#plt.imshow(h0)
#plt.colorbar()

eta0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
u0 = np.zeros((ny+2, nx+1+2), dtype=np.float32, order='C');
v0 = np.zeros((ny+1+2, nx+2), dtype=np.float32, order='C');

#Create bump in domain for testing
x_center = dx*nx*0.5
y_center = dy*ny*0.5
#makeCentralBump(eta0, 0.0, nx, ny, dx, dy, ghosts)
    

#Initialize simulator
reload(CTCS)
# NOTE! offset_x and offset_y is chosen to produce 'reasonable' lonlat-values using the bogus
# projection variable currently added by SimNetCDFWriter.
sim = CTCS.CTCS(cl_ctx, \
                h0, eta0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, A, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions, \
                ignore_ghostcells=True, \
                write_netcdf=False, \
                offset_x=-2000000.0, offset_y=-1000000.0)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

ghosts = [-1,1,-1,1]

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta0[1:-1, 1:-1], u0[1:-1, :], v0[:, 1:-1])

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[1:-1, 1:-1], u1[1:-1, :], v1[:, 1:-1]);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t) + ", max(eta): " + str(np.max(eta1))
        #fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim
    
#for i in range(100):
#    if (i>0):
#        t = sim.step(10.0)
#    else:
#        t = 0.0
#    eta1, u1, v1 = sim.download()
#
#    print "i: " + str(i) + " t: " + str(t)

In [ ]:
# Close nc-file
sim.cleanUp()

In [ ]:
# Close nc-file
sim.cleanUp()

h0 = np.ones((ny+5, nx+5), dtype=np.float32, order='C')*5 

# Bottom with a bump:
#h0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
#addBump(h0, nx, ny, dx, dy, 0.5, 0.5, 600, ghosts)
#h0 = (5.0 - h0*4.2)
#fig = plt.figure(figsize=(4,4))
#plt.imshow(h0)
#plt.colorbar()

eta0 = np.zeros((ny+4, nx+4), dtype=np.float32, order='C');
u0 = np.zeros((ny+4, nx+4), dtype=np.float32, order='C');
v0 = np.zeros((ny+4, nx+4), dtype=np.float32, order='C');

#Create bump in domain for testing
x_center = dx*nx*0.5
y_center = dy*ny*0.5
#makeCentralBump(eta0, 0.0, nx, ny, dx, dy, ghosts)
    
f = 0.0
#Initialize simulator
reload(CDKLM16)
# NOTE! offset_x and offset_y is chosen to produce 'reasonable' lonlat-values using the bogus
# projection variable currently added by SimNetCDFWriter.
sim = CDKLM16.CDKLM16(cl_ctx, \
                      eta0, u0, v0, h0,\
                      nx, ny, \
                      dx, dy, dt, \
                      g, f, r, \
                      wind_stress=wind, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

ghosts = [-2,2,-2,2]

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta0[2:-2, 2:-2], u0[2:-2, 2:-2], v0[2:-2, 2:-2])

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[2:-2, 2:-2], u1[2:-2, 2:-2], v1[2:-2, 2:-2]);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t) + ", max(eta): " + str(np.max(eta1)) + \
        ", (max(hu), max(hv)): " + str((np.max(u1), np.max(v1)))
        #fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim
    